# Real-Time Stream Viewer (HTTP)
the following function responds to HTTP requests with the list of last 10 processed twitter messages + sentiments in reverse order (newest on top), it reads records from the enriched stream, take the recent 10 messages, and reverse sort them. the function is using nuclio context to store the last results and stream pointers for max efficiency.<br> 

The code is automatically converted into a nuclio (serverless) function and and respond to HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations, and to deploy functions automatically onto a cluster.


## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that don't need to be copied to the function

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [3]:
%%nuclio config 
# ask for a specific (fixed) API port
spec.triggers.web.kind = "http"
spec.triggers.web.attributes.port = 30100
# define the function base docker image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.web.kind to 'http'
%nuclio: setting spec.triggers.web.attributes.port to 30099
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [ ]:
%nuclio cmd pip install git+https://github.com/yaronha/v3io-py-http.git

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [4]:
import v3io
import base64
import json
import os

v3 = v3io.V3io(container='bigdata')

def handler(context, event):


    resp = v3.getrecords('stock_stream','0',context.next_location,10)
    json_resp = resp.json()
    context.next_location = json_resp['NextLocation']

    context.logger.info('location: %s', context.next_location)

    for rec in json_resp['Records'] :
        rec_data = base64.b64decode(rec['Data']).decode('utf-8')
        rec_json = json.loads(rec_data)['text']
        context.data += [rec_json]

    context.data = context.data[-10:]

    return context.Response(body=json.dumps(context.data[::-1]),
                            headers={'Access-Control-Allow-Origin': '*'},
                            content_type='text/plain',
                            status_code=200)


def init_context(context):
    resp = v3.seek('stock_stream','0','EARLIEST')
    context.next_location = resp.json()['Location']
    context.data = []

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [5]:
# nuclio: ignore
# create a test event and invoke the function locally 
init_context(context)
event = nuclio.Event(body='')
handler(context, event)

Python> 2019-03-05 11:33:24,910 [info] location: AQAAAAEAAAAAAECLAQAAAA==


[20190305T113324 - nuclio_sdk - INFO] location: AQAAAAEAAAAAAECLAQAAAA==


Response(headers={'Access-Control-Allow-Origin': '*'}, body='["RT @Sophiemcneill: Just in - @Google, siding with Saudi Arabia, refuses to remove widely-criticized government app which lets men track wom\\u2026", "Diversity and Inclusion Consultants @prdgm, whose CEO @joelle_emerson criticized @Google for going for \\"Equality\\" r\\u2026 https://t.co/Ih7l8EX4mu", "RT @MyWhiteNinja_: OOF https://t.co/3BsS1O5UFI", "RT @AAPPres: Right now, pediatricians are watching our worst fears realized as measles outbreaks spread across the country. My home state o\\u2026", "RT @MohapatraHemant: So @lyft is paying $8m/mo to @AWS -- almost $100m/yr! Each ride costs $.14 in AWS rent. I keep hearing they could buil\\u2026"]', status_code=200, content_type='text/plain')

## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [8]:
%nuclio deploy -p stocks -c stream-view

%nuclio: [nuclio.deploy] 2019-03-05 11:37:10,433 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-03-05 11:37:12,464 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-03-05 11:37:12,464 (info) Build complete
%nuclio: [nuclio.deploy] 2019-03-05 11:37:15,491 done updating stream-viewer, function address: 3.122.204.208:30100
%nuclio: function deployed


In [9]:
# nuclio: ignore
# test the new API end point, take the address from the deploy log above
!curl 3.122.204.208:30100

["RT @Sophiemcneill: Just in - @Google, siding with Saudi Arabia, refuses to remove widely-criticized government app which lets men track wom\u2026", "Diversity and Inclusion Consultants @prdgm, whose CEO @joelle_emerson criticized @Google for going for \"Equality\" r\u2026 https://t.co/Ih7l8EX4mu", "RT @MyWhiteNinja_: OOF https://t.co/3BsS1O5UFI", "RT @AAPPres: Right now, pediatricians are watching our worst fears realized as measles outbreaks spread across the country. My home state o\u2026", "RT @MohapatraHemant: So @lyft is paying $8m/mo to @AWS -- almost $100m/yr! Each ride costs $.14 in AWS rent. I keep hearing they could buil\u2026"]